This notebook analyzes on-chain USDC activity on Ethereum over a recent 30-day window.
The goal is to demonstrate how stablecoin transaction data can be collected, cleaned, and analyzed using public blockchain APIs, and to highlight key limitations of high-frequency on-chain data.

Transaction data is retrieved using the Etherscan API for ERC-20 token transfers involving the USDC contract on Ethereum. Due to API constraints, the analysis focuses on a recent 30-day window.

In [1]:
import pandas as pd

df = pd.read_csv("../data/usdc_raw_30d.csv")
df.shape


(10000, 21)

In [2]:
# Data Cleaning

# Convert timestamp to datetime
df["timeStamp"] = pd.to_datetime(df["timeStamp"], unit="s")

# Create date column
df["date"] = df["timeStamp"].dt.date

# Convert USDC value from on-chain units (6 decimals)
df["usdc_amount"] = df["value"].astype(float) / 1_000_000

df[["timeStamp", "date", "usdc_amount"]].head()


,timeStamp,date,usdc_amount
0,2025-12-18 19:15:23,2025-12-18,95.000000
1,2025-12-18 19:15:23,2025-12-18,638781.773539
2,2025-12-18 19:15:23,2025-12-18,11724.703951
3,2025-12-18 19:15:23,2025-12-18,296.816675
4,2025-12-18 19:15:23,2025-12-18,468.101294


In [3]:
# checking the timeframe of our fetched data
df["date"].min(), df["date"].max()


(datetime.date(2025, 12, 18), datetime.date(2025, 12, 18))

Due to the high transaction frequency of major stablecoins such as USDC, standard blockchain explorer APIs return only the most recent transactions, truncating longer time windows. 

To extend coverage beyond a single day, I implement a block-by-day querying strategy using the Etherscan API. While this approach allows recovery of multiple days of USDC transactions, high-activity days remain truncated due to API result limits on the free API plan.